# Mercedes-Benz Greener Manufacturing

In [12]:
import pandas as pd
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')
print(train.shape)
print(test.shape)

(4209, 378)
(4209, 377)


In [13]:
train.head()

,ID,y,X0,X1,X2,X3,X4,X5,X6,X8,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,130.81,k,v,at,a,d,u,j,o,...,0,0,1,0,0,0,0,0,0,0
1,6,88.53,k,t,av,e,d,y,l,o,...,1,0,0,0,0,0,0,0,0,0
2,7,76.26,az,w,n,c,d,x,j,x,...,0,0,0,0,0,0,1,0,0,0
3,9,80.62,az,t,n,f,d,x,l,e,...,0,0,0,0,0,0,0,0,0,0
4,13,78.02,az,v,n,f,d,h,d,n,...,0,0,0,0,0,0,0,0,0,0


In [14]:
total=pd.concat([train,test])

In [15]:
from sklearn.preprocessing import LabelEncoder
cat_cols=train.select_dtypes(include=object).columns
for col in cat_cols:
    LB=LabelEncoder()
    total[col]=LB.fit_transform(total[col])

In [16]:
total.head()

,ID,y,X0,X1,X2,X3,X4,X5,X6,X8,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,130.81,37,23,20,0,3,27,9,14,...,0,0,1,0,0,0,0,0,0,0
1,6,88.53,37,21,22,4,3,31,11,14,...,1,0,0,0,0,0,0,0,0,0
2,7,76.26,24,24,38,2,3,30,9,23,...,0,0,0,0,0,0,1,0,0,0
3,9,80.62,24,21,38,5,3,30,11,4,...,0,0,0,0,0,0,0,0,0,0
4,13,78.02,24,23,38,5,3,14,3,13,...,0,0,0,0,0,0,0,0,0,0


## Minimum personal best

In [6]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import numpy as np
train1=total[total.y.notna()]
tx,vx,ty,vy=train_test_split(train1.drop('y',axis=1),train1.y)
XGBR1=XGBRegressor(n_estimators=1000,learning_rate=0.05,n_jobs=6)
print(np.mean(cross_val_score(XGBR1,train1.drop('y',axis=1),train1.y,scoring='neg_mean_absolute_error')))
LR=LinearRegression()
print(np.mean(cross_val_score(LR,train1.drop('y',axis=1),train1.y,scoring='neg_mean_absolute_error')))

-8.322828817157937


In [22]:
from sklearn.linear_model import LinearRegression
LR=LinearRegression()
print(np.mean(cross_val_score(LR,train1.drop('y',axis=1),train1.y,scoring='neg_mean_absolute_error')))

-5.858586352235579


## SelectKBest

In [53]:
from sklearn.feature_selection import SelectKBest,f_regression
import warnings
warnings.filterwarnings('ignore')

#To prevent overfitting,select best 20 festures
SKB=SelectKBest(f_regression,k=110)
train2=SKB.fit_transform(train1.drop('y',axis=1),train1.y)

#new dataframe where except selected features all other are 0
sel_feat=pd.DataFrame(SKB.inverse_transform(train2),index=train1.index,columns=train1.drop('y',axis=1).columns)
sel_cols=sel_feat.columns[sel_feat.var()!=0]
train3=train1[sel_cols]

#XGBRegressor on these 20 features
#XGBR2=XGBRegressor(n_estimators=1000,learning_rate=0.05,n_jobs=6)
#print(np.mean(cross_val_score(XGBR2,train3,train1.y,scoring='neg_mean_absolute_error')))

#LinearRegression
LR1=LinearRegression()
print(np.mean(cross_val_score(LR1,train3,train1.y,scoring='neg_mean_absolute_error')))

-5.535410131358775


## Priciple Component Analysis (PCA)

In [40]:
from sklearn.decomposition import PCA
pca = PCA(n_components=110)
train4=pd.DataFrame(pca.fit_transform(train1.drop(['y'],axis=1)))

#XGBRegressor on these 20 features
#XGBR3=XGBRegressor(n_estimators=1000,learning_rate=0.05,n_jobs=6)
#print(np.mean(cross_val_score(XGBR3,train4,train1.y,scoring='neg_mean_absolute_error')))

#LinearRegression
LR2=LinearRegression()
print(np.mean(cross_val_score(LR2,train4,train1.y,scoring='neg_mean_absolute_error')))

-5.688131880083972


## Random Forest Regressor

In [61]:
from sklearn.ensemble import RandomForestRegressor 
RFR = RandomForestRegressor(n_estimators = 200, random_state = 0) 
print(np.mean(cross_val_score(RFR,train3,train1.y,scoring='neg_mean_absolute_error')))

-5.975807072678871


## Ridge

In [59]:
from sklearn.linear_model import Ridge
RR= Ridge(alpha=1.0,normalize=True, max_iter=1e5)
print(np.mean(cross_val_score(RR,train3,train1.y,scoring='neg_mean_absolute_error')))

-5.757398938366433


In [69]:
test1=total[total.y.isna()]
df=pd.DataFrame(test1.ID)
LR1.fit(train3,train1.y)
df.insert(1,'y',LR1.predict(test1[sel_cols]))
df.to_csv('SC1.csv',index=False)

In [70]:
df

,ID,y
0,1,86.949889
1,2,104.194070
2,3,85.536715
3,4,79.653987
4,5,111.737756
...,...,...
4204,8410,106.501390
4205,8411,94.133700
4206,8413,93.558100
4207,8414,109.024351
